In [ ]:
# Setup
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Dict, List, Tuple

from core.multi_asset_loader import load_assets
from core.multi_asset_signal import SingleAssetWrapper
from signals.momentum import MomentumSignalV2
from core.portfolio_manager import PortfolioConfig
from backtest.backtest_engine import run_multi_asset_backtest

print("✅ Imports successful")

---

## Walk-Forward Methodology

```
┌────────────────────────────────────────────────────────┐
│                  ROLLING WINDOW                        │
└────────────────────────────────────────────────────────┘

Window 1:
├─ Train: 2010-2012 (2 years) → Pick best params
└─ Test:  2013      (1 year)  → Trade with those params

Window 2:
├─ Train: 2011-2013 (2 years) → Pick NEW best params
└─ Test:  2014      (1 year)  → Trade with NEW params

Window 3:
├─ Train: 2012-2014 (2 years) → Pick NEW best params
└─ Test:  2015      (1 year)  → Trade with NEW params

...

Key Rules:
1. NEVER use test data in training
2. NEVER reuse parameters from previous window
3. Optimize ONLY on train period
4. Trade ONLY with those params in test period
```

---

## Step 1: Define Walk-Forward Splits

**Goal:** Split data into non-overlapping train/test windows

In [ ]:
def create_walk_forward_splits(
    start_date: str,
    end_date: str,
    train_years: int = 2,
    test_years: int = 1
) -> List[Dict]:
    """
    Create rolling windows for walk-forward testing.
    
    Returns list of dicts:
    [
        {'train_start': '2010-01-01', 'train_end': '2011-12-31',
         'test_start': '2012-01-01', 'test_end': '2012-12-31'},
        ...
    ]
    """
    splits = []
    current_train_start = pd.Timestamp(start_date)
    final_date = pd.Timestamp(end_date)
    
    while True:
        # Train period
        train_end = current_train_start + pd.DateOffset(years=train_years)
        
        # Test period
        test_start = train_end + pd.DateOffset(days=1)
        test_end = test_start + pd.DateOffset(years=test_years)
        
        # Stop if we exceed final date
        if test_end > final_date:
            break
        
        splits.append({
            'train_start': current_train_start.strftime('%Y-%m-%d'),
            'train_end': train_end.strftime('%Y-%m-%d'),
            'test_start': test_start.strftime('%Y-%m-%d'),
            'test_end': test_end.strftime('%Y-%m-%d')
        })
        
        # Roll forward by test period
        current_train_start = test_start
    
    return splits

# Create splits
splits = create_walk_forward_splits(
    start_date='2020-01-01',
    end_date='2024-12-31',
    train_years=2,
    test_years=1
)

print("\n📅 Walk-Forward Splits:")
print(f"\nTotal Windows: {len(splits)}\n")
for i, split in enumerate(splits, 1):
    print(f"Window {i}:")
    print(f"  Train: {split['train_start']} to {split['train_end']}")
    print(f"  Test:  {split['test_start']} to {split['test_end']}")
    print()

---

## Step 2: Parameter Grid

**Define parameters to test in each training window**

In [ ]:
# Parameter grid for momentum strategy
param_grid = {
    'lookback': [60, 90, 120, 150],
    'sma_filter': [150, 200, 250]
}

# Generate all combinations
from itertools import product

param_combinations = [
    {'lookback': lb, 'sma_filter': sma}
    for lb, sma in product(param_grid['lookback'], param_grid['sma_filter'])
]

print("\n⚙️ Parameter Grid:")
print(f"\nTotal Combinations: {len(param_combinations)}\n")
print("Sample combinations:")
for params in param_combinations[:5]:
    print(f"  Lookback={params['lookback']}, SMA={params['sma_filter']}")
print("  ...")

---

## Step 3: Train - Optimize on Training Data

**For ONE window, test all parameter combinations on training period**

In [ ]:
def optimize_parameters(
    train_split: Dict,
    param_combinations: List[Dict],
    tickers: List[str]
) -> Tuple[Dict, pd.DataFrame]:
    """
    Test all parameter combinations on training data.
    Return best params and all results.
    """
    print(f"\n🔍 Optimizing on {train_split['train_start']} to {train_split['train_end']}")
    
    # Load training data
    train_prices = load_assets(
        tickers=tickers,
        start_date=train_split['train_start'],
        end_date=train_split['train_end']
    )
    
    results = []
    
    for params in param_combinations:
        # Generate signals with these params
        signal_gen = MomentumSignalV2(
            lookback=params['lookback'],
            sma_filter=params['sma_filter']
        )
        multi_signal = SingleAssetWrapper(signal_gen)
        signals = multi_signal.generate(train_prices)
        
        # Run backtest
        config = PortfolioConfig(initial_capital=100000)
        result = run_multi_asset_backtest(train_prices, signals, config)
        
        # Store results
        results.append({
            'lookback': params['lookback'],
            'sma_filter': params['sma_filter'],
            'sharpe': result.metrics['sharpe_ratio'],
            'total_return': result.metrics['total_return'],
            'max_dd': result.metrics['max_drawdown'],
            'win_rate': result.metrics['win_rate']
        })
    
    results_df = pd.DataFrame(results).sort_values('sharpe', ascending=False)
    best_params = results_df.iloc[0][['lookback', 'sma_filter']].to_dict()
    
    print(f"\n✅ Best Parameters:")
    print(f"   Lookback: {best_params['lookback']}")
    print(f"   SMA Filter: {best_params['sma_filter']}")
    print(f"   Sharpe: {results_df.iloc[0]['sharpe']:.2f}")
    
    return best_params, results_df

# Example: Optimize first window
TICKERS = ['ES=F', 'GC=F', 'NQ=F']
best_params, train_results = optimize_parameters(splits[0], param_combinations[:3], TICKERS)  # Testing 3 for speed

print("\n📊 Top 3 Parameter Sets:")
print(train_results.head(3).to_string(index=False))

---

## Step 4: Test - Evaluate on Out-of-Sample Period

**Use ONLY the best params from training. Trade on unseen test data.**

In [ ]:
def test_parameters(
    test_split: Dict,
    best_params: Dict,
    tickers: List[str]
) -> Dict:
    """
    Test parameters on out-of-sample data.
    """
    print(f"\n🧪 Testing on {test_split['test_start']} to {test_split['test_end']}")
    print(f"   Using params: Lookback={best_params['lookback']}, SMA={best_params['sma_filter']}")
    
    # Load test data
    test_prices = load_assets(
        tickers=tickers,
        start_date=test_split['test_start'],
        end_date=test_split['test_end']
    )
    
    # Generate signals with BEST params
    signal_gen = MomentumSignalV2(
        lookback=int(best_params['lookback']),
        sma_filter=int(best_params['sma_filter'])
    )
    multi_signal = SingleAssetWrapper(signal_gen)
    signals = multi_signal.generate(test_prices)
    
    # Run backtest
    config = PortfolioConfig(initial_capital=100000)
    result = run_multi_asset_backtest(test_prices, signals, config)
    
    print(f"\n✅ Out-of-Sample Results:")
    print(f"   Sharpe: {result.metrics['sharpe_ratio']:.2f}")
    print(f"   Total Return: {result.metrics['total_return']:.2%}")
    print(f"   Max DD: {result.metrics['max_drawdown']:.2%}")
    
    return result.metrics

# Test on OOS period
oos_metrics = test_parameters(splits[0], best_params, TICKERS)

---

## Step 5: Compare IS vs OOS Performance

**The moment of truth: Did the strategy overfit?**

In [ ]:
# Get IS performance (best params on training data)
is_sharpe = train_results.iloc[0]['sharpe']
is_return = train_results.iloc[0]['total_return']

# OOS performance
oos_sharpe = oos_metrics['sharpe_ratio']
oos_return = oos_metrics['total_return']

print("\n📊 IS vs OOS Comparison:")
print(f"\n{'Metric':<20} {'In-Sample':>15} {'Out-of-Sample':>15} {'Degradation':>15}")
print("-" * 70)

sharpe_deg = (oos_sharpe - is_sharpe) / is_sharpe if is_sharpe != 0 else 0
return_deg = (oos_return - is_return) / is_return if is_return != 0 else 0

print(f"{'Sharpe Ratio':<20} {is_sharpe:>15.2f} {oos_sharpe:>15.2f} {sharpe_deg:>14.1%}")
print(f"{'Total Return':<20} {is_return:>14.2%} {oos_return:>14.2%} {return_deg:>14.1%}")

print("\n💡 Interpretation:")
if abs(sharpe_deg) < 0.30:
    print("   ✅ GOOD: Performance degradation < 30%")
    print("   Strategy appears robust!")
elif abs(sharpe_deg) < 0.50:
    print("   ⚠️ MODERATE: Performance degradation 30-50%")
    print("   Some overfitting, but acceptable")
else:
    print("   ❌ BAD: Performance degradation > 50%")
    print("   Severe overfitting detected!")

---

## Step 6: Full Walk-Forward Engine

**Run ALL windows and aggregate OOS results**

In [ ]:
def run_walk_forward(
    splits: List[Dict],
    param_combinations: List[Dict],
    tickers: List[str]
) -> pd.DataFrame:
    """
    Run full walk-forward test.
    Returns DataFrame with all window results.
    """
    all_results = []
    
    for i, split in enumerate(splits, 1):
        print(f"\n{'='*60}")
        print(f"WINDOW {i}/{len(splits)}")
        print(f"{'='*60}")
        
        # Train: Optimize parameters
        best_params, train_results = optimize_parameters(split, param_combinations, tickers)
        
        # Test: Evaluate OOS
        oos_metrics = test_parameters(split, best_params, tickers)
        
        # Store results
        all_results.append({
            'window': i,
            'test_start': split['test_start'],
            'test_end': split['test_end'],
            'best_lookback': best_params['lookback'],
            'best_sma': best_params['sma_filter'],
            'is_sharpe': train_results.iloc[0]['sharpe'],
            'oos_sharpe': oos_metrics['sharpe_ratio'],
            'oos_return': oos_metrics['total_return'],
            'oos_max_dd': oos_metrics['max_drawdown']
        })
    
    return pd.DataFrame(all_results)

# Note: This would take a while to run with full param grid
# For demo, we'll show the structure
print("\n🔧 Walk-Forward Engine Structure:")
print("\nFor production use:")
print("  1. Run run_walk_forward(splits, param_combinations, TICKERS)")
print("  2. Analyze aggregate OOS performance")
print("  3. Check consistency across windows")
print("  4. Deploy if OOS Sharpe > threshold (e.g., 0.5)")

---

## Step 7: Aggregate OOS Equity Curve

**Stitch together OOS periods to see true performance**

In [ ]:
# Visualization of what aggregate OOS looks like
print("\n📈 Aggregate OOS Equity Curve:")
print("\nConcept:")
print("  Window 1 OOS: $100k → $110k (Jan-Dec 2022)")
print("  Window 2 OOS: $110k → $118k (Jan-Dec 2023)")
print("  Window 3 OOS: $118k → $125k (Jan-Dec 2024)")
print("\n  Final OOS Return: +25%")
print("  This is your TRUE expected performance!")

print("\n💡 Why This Matters:")
print("  ✅ Never used future data")
print("  ✅ Parameters re-optimized each window")
print("  ✅ Realistic transaction costs included")
print("  ✅ Can compare to benchmark (buy-and-hold)")

---

## 🎓 Key Takeaways

### Walk-Forward Testing Rules:

```
DO:
✅ Optimize on training period only
✅ Test on future unseen data
✅ Re-optimize for each window
✅ Use consistent train/test split ratio
✅ Report aggregate OOS metrics

DON'T:
❌ Use test data in training
❌ Cherry-pick good windows
❌ Change parameters during test period
❌ Ignore transaction costs
❌ Report in-sample results as final
```

### Performance Degradation:

| IS Sharpe | OOS Sharpe | Degradation | Status |
|-----------|------------|-------------|--------|
| 2.0 | 1.8 | -10% | ✅ Excellent |
| 2.0 | 1.4 | -30% | ✅ Good |
| 2.0 | 1.0 | -50% | ⚠️ Acceptable |
| 2.0 | 0.5 | -75% | ❌ Overfit |
| 2.0 | -0.5 | -125% | ❌❌ Total failure |

### Why Parameters Change:

Market conditions evolve! What worked in 2020 (low vol, QE) may not work in 2022 (high vol, rate hikes).

**Adaptive optimization** keeps strategy relevant.

### Production Workflow:

```python
# 1. Development
results = run_walk_forward(splits, params, tickers)
if results['oos_sharpe'].mean() > 0.5:
    print("Strategy approved for production!")

# 2. Deployment
# - Use latest window's best params
# - Re-optimize monthly or quarterly
# - Monitor live vs OOS performance
```

### Critical Insights:

1. **OOS is the ONLY metric that matters**
   - In-sample tells you nothing about future
   - Always report OOS Sharpe, never IS

2. **Some degradation is normal**
   - 20-30% drop IS→OOS is expected
   - 0% degradation suggests data leakage!

3. **Consistency > Peak Performance**
   - Prefer 0.8 Sharpe across all windows
   - Over 2.0 Sharpe in one window, 0.2 in others

---

## ✅ Congratulations!

You now understand:
1. ✅ Basic backtest mechanics
2. ✅ Portfolio manager orchestration
3. ✅ Risk management integration
4. ✅ Full system pipeline
5. ✅ Walk-forward validation methodology

**You're ready to:**
- Build new strategies with confidence
- Properly validate before deploying
- Avoid overfitting traps
- Present results professionally

## 🚀 Next Steps:

1. **Implement walk-forward in production**
   - Create `walk_forward_engine.py`
   - Add to backtest folder
   - Test existing momentum strategy

2. **Research new signals**
   - Pairs trading (cointegration)
   - Energy signals (contango/backwardation)
   - Regime detection (HMM, vol regimes)

3. **Build CV projects**
   - Each with proper walk-forward validation
   - Document methodology clearly
   - Show OOS performance only

---

**Happy Trading! 📈**